In [9]:
import psycopg2
import pandas as pd

# Database connection details
db_config = {
    'dbname': 'Car-Rentals ',
    'user': '', #Fill in with username
    'password': '', #Fill in with password
    'host': 'localhost',  # Or your database host
    'port': 5432          # Default PostgreSQL port
}

# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(**db_config)
    print("Database connected successfully!")
except Exception as e:
    print(f"Error: {e}")

Database connected successfully!


In [10]:
def run_query(query):
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            # Fetch all rows and column names
            rows = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            return pd.DataFrame(rows, columns=columns)
    except Exception as e:
        print(f"Error: {e}")
        return None

In [11]:
# --Rank the employees based on number of rental agreements created--
query = """
SELECT
    RANK() OVER (ORDER BY COUNT(r.rentalid) DESC) AS rank,
    e.employeeid AS employee_id,
    e.first_name AS first_name,
    e.last_name AS last_name,
    COUNT(r.rentalid) AS rental_count
FROM
    employee AS e
        LEFT JOIN
    rental_agreement AS r
    ON
        e.employeeid = r.employeeid
GROUP BY
    e.employeeid, e.first_name, e.last_name
ORDER BY
    rank;
"""

# Run the query and display results
df = run_query(query)
if df is not None:
    display(df)



,rank,employee_id,first_name,last_name,rental_count
0,1,14,Jessica,Johnson,12
1,2,18,Jessica,Brown,10
2,3,11,Emma,Rodriguez,7
3,4,7,Jessica,Davis,6
4,4,6,Michael,Garcia,6
5,4,13,Michael,Brown,6
6,4,5,David,Johnson,6
7,4,4,Emma,Jones,6
8,9,1,Sarah,Garcia,5
9,9,8,Emily,Martinez,5


In [5]:
# --Rank the employees based on order of revenue generated--
query = """
SELECT
    e.employeeid AS employee_id,
    e.first_name || ' ' || e.last_name AS employee_name,
    COUNT(r.rentalid) AS total_rentals,
    ROUND(CAST(SUM((r.return_date - r.start_date) * v.price_per_day) AS NUMERIC), 2) AS total_revenue
FROM
    rental_agreement AS r
INNER JOIN
    vehicle AS v
ON
    r.vehicleid = v.vehicleid
INNER JOIN
    employee AS e
ON
    r.employeeid = e.employeeid
GROUP BY
    e.employeeid, e.first_name, e.last_name
ORDER BY
    total_revenue DESC;
"""

# Run the query and display results
df = run_query(query)
if df is not None:
    display(df)

,employee_id,employee_name,total_rentals,total_revenue
0,14,Jessica Johnson,12,19730.31
1,6,Michael Garcia,6,18749.90
2,18,Jessica Brown,10,14589.58
3,11,Emma Rodriguez,7,14368.90
4,7,Jessica Davis,6,12575.23
5,1,Sarah Garcia,5,12382.80
6,5,David Johnson,6,11368.70
7,4,Emma Jones,6,9537.21
8,3,Daniel Rodriguez,5,9391.34
9,17,Emily Jones,3,8806.00


In [14]:
# -- What top 10 car has generated the most revenue overall--
query = """
SELECT
    v.vehicleid,
    v.model,
    v.make,
    v.year,
    SUM(r.return_date - r.start_date) AS total_rental_days,
    v.price_per_day,
    ROUND(SUM((r.return_date - r.start_date) * v.price_per_day)::numeric, 2) AS total_revenue
FROM
    rental_agreement AS r
        JOIN
    vehicle AS v
    ON
        r.vehicleid = v.vehicleid
GROUP BY
    v.vehicleid, v.model, v.make, v.year, v.price_per_day
ORDER BY
    total_revenue DESC
LIMIT 10;
"""

# Run the query and display results
df = run_query(query)
if df is not None:
    display(df)


,vehicleid,model,make,year,total_rental_days,price_per_day,total_revenue
0,38,Escape,Ford,2010,64,194.65,12457.60
1,27,Model S,Tesla,2006,83,139.63,11589.29
2,20,Model X,Tesla,2008,57,185.36,10565.52
3,44,Model 3,Tesla,2003,62,161.01,9982.62
4,47,CR-V,Honda,2010,76,123.98,9422.48
5,8,RAV4,Toyota,2015,58,153.44,8899.52
6,41,Model 3,Tesla,2007,48,180.15,8647.20
7,32,Accord,Honda,2020,46,159.69,7345.74
8,26,CR-V,Honda,2018,50,128.30,6415.00
9,46,Escape,Ford,2014,38,160.62,6103.56


In [15]:
# --How many cars are rented on a per month basis--

query = """SELECT
    TO_CHAR(start_date, 'Month') AS month_name,
    COUNT(*) AS rental_count
FROM
    rental_agreement
GROUP BY
    TO_CHAR(start_date, 'Month')
ORDER BY
    MIN(EXTRACT(MONTH FROM start_date));
"""

# Run the query and display results
df = run_query(query)
if df is not None:
    display(df)


,month_name,rental_count
0,January,5
1,February,13
2,March,9
3,April,11
4,May,8
5,June,8
6,July,8
7,August,7
8,September,10
9,October,8


In [ ]:
# Close the connection when done
connection.close()